# ファイル操作（NEW I/O API）

Javaのファイル操作はjava.io.Fileで行うことが主流でした。  
Java7でjava.nio.fileが導入されたことにより、ファイル操作方法ががらりと変わりました。  
nio＝NEW I/O APIです  

[java.io.Fileでの操作](01-10_File.ipynb)で操作した内容と同等の処理を試してみましょう。

## java.nio.fileの説明
|クラス|説明|
|:--|:--|
|java.nio.file.Path|ファイル・システム内のファイルを特定するためのインタフェース。|
|java.nio.file.Paths|パス文字列またはURIを変換することによってPathを返すクラス|
|java.nio.file.Files|Pathインターフェース、及びPathsクラスでファイルパスを指定し、Filesクラスでファイルを操作するイメージ。<br>Filesの各メソッドは、基本的にPathを引数に取る。|

# Filesクラス、Pathsクラス

In [ ]:
import java.nio.file.*;
import java.util.stream.*;
import java.nio.charset.*;

## ファイルを新規に作成する
ファイルを作成するにはFilesクラスのcreateFileメソッドを使用します。  
まず、Pathsクラスのgetメソッドを使用してPathのインスタンスを取得します。  
createfileメソッドの引数にPathインスタンスを渡してファイルを新規に作成します。  
新しい空のファイルを作成し、ファイルがすでに存在する場合は失敗します。  

ディレクトリを作成する場合はcreateDirectoryメソッドを使用します。

In [ ]:
// 作成するファイル名を指定する
Path file = Paths.get("./1_10_nio_files_createFile.txt");
Files.createFile(file);

## ファイル名の変更・ファイルの移動をする
ファイル名を変更や移動を行うには、Filesクラスのmoveメソッドを使用します。

In [ ]:
// ファイル名を変更するPathの指定
Path from = Paths.get("./1_10_nio_files_createFile.txt");
// 変更後のファイルPathの指定 例えば ""../1_10_nio_files_createFile.txt"などとすれば、1つ上の階層に移動できる
Path to = Paths.get("./1_10_nio_files_renamedFile.txt");

Files.move(from, to);

## ファイルの読み込み可否判定
ファイルを読み込む際には権限がないなどに理由で読み込みに失敗することがあります。<br>
isReadableメソッドを使用することでファイルを読み込むことができるかを事前に知ることができます。

In [ ]:
Path file = Paths.get("./1_10_nio_files_renamedFile.txt");
 
//ファイルの読み込み可否判定
if (Files.isReadable(file)){
    System.out.println(file + "は読み込み可");
}else{
    System.out.println(file + "は読み込み不可");
}

## ファイルへの書き込み可否判定
ファイルの読み込みと同様、権限がないなどの理由で書き込みに失敗することがあります。<br>
isWritableメソッドを使用することでファイルに書き込むことができるかを事前に知ることができます。

In [ ]:
Path file = Paths.get("./1_10_nio_files_renamedFile.txt");
 
//ファイルの書き込み可否判定
if (Files.isWritable(file)){
    System.out.println(file + "は書き込み可");
}else{
    System.out.println(file + "は書き込み不可");
}

## ディレクトリ・ファイルの存在を確認する
指定したディレクトリやファイルが存在するか確認するにはexistsメソッドを使用します。

In [ ]:
Path file = Paths.get("./1_10_nio_files_renamedFile.txt");
        
//ディレクトリ、ファイルの存在を確認する
if (Files.exists(file)){
    System.out.println(file + "は存在する");
}else{
    System.out.println(file + "は存在しない");
}        

## ファイルリストを取得する
指定したディレクトリのファイルの一覧を取得するにはlistメソッドを使用します。

In [ ]:
// カレントディレクトリを指定する
Path dir = Paths.get("./");
        
// listメソッドを使用して一覧を取得する
try(Stream<Path> stream = Files.list(dir)){
  stream.forEach(path -> {
    System.out.println(path);
  });
}

## ファイルを削除する
ファイルを削除するにはdeleteメソッドを使用します。

In [ ]:
Path file = Paths.get("./1_10_nio_files_renamedFile.txt");
Files.delete(file);

## ファイルに書き込む 
Filesクラスでファイルに書き込む際にはwriteメソッドを使用します。<br>
このメソッドは、すべてのバイトが書き込まれた(あるいは、入出力エラーまたは他の実行時例外がスローされた)ときに必ずそのファイルがクローズされるようにします。

In [ ]:
String str = ("java");

Path file = Paths.get("./1_10_nio_files.txt");
try {
  Files.write(file, str.getBytes());
  // Files.writeString(file, "java"); // Java 11以降の場合
} catch (Exception e) {
  e.getStackTrace();
}

複数行の書込みがある場合はFiles.newBufferedWriter(Path, Charset, OpenOption...) から返される java.io.BufferedWriterを使用します。  
try-with-resource形式で記載することで、finallyブロックでcloseメソッドを呼び出さなくても自動でcloseをしてくれるようになります。
[詳細はこちら](./01-05_Exception.ipynb)

In [ ]:
List<String> lines = new ArrayList<String>();
lines.add("java");
lines.add("study");
lines.add("writeFile");

Path file = Paths.get("./1_10_nio_files.txt");

try (BufferedWriter bw = Files.newBufferedWriter(file, StandardCharsets.UTF_8)) {  // 文字コードUTF-8指定
    for (String line : lines) {
        bw.write(line);
        bw.newLine();
    }
} catch (IOException e) {
    e.printStackTrace();
}

## ファイルを読み込む
ファイルの読み込みはreadAllBytesメソッドを使用します。  
読み込みと同時に書込みをすることでコピーを実現しています。

In [ ]:
Path file = Paths.get("./1_10_nio_files.txt");
try {
  Files.write(Paths.get("./1_10_nio_files_copy.txt"), Files.readAllBytes(file));  
} catch (Exception e) {
  e.getStackTrace();
}

 # 問題1
 先程作成して書き込みを行った1_10_nio_files.txt、1_10_nio_files_copy.txtの2つのファイルに「practice1」という文字列を`追記`して表示してみましょう。

# 問題2
今回作成した1_10_nio_files.txt、1_10_nio_files_copy.txtを削除してみましょう。